In [ ]:
!pip install mediapipe
!pip install ultralytics

In [ ]:
import torch

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

In [ ]:
import os
print(os.getcwd())

In [ ]:
def piece2Num(piece):
    cases = {
        "black-bishop": lambda: 201,
        "black-king": lambda: 202,
        "black-knight": lambda: 203,
        "black-pawn": lambda: 204,
        "black-queen": lambda: 205,
        "black-rook": lambda: 206,
        "white-bishop": lambda: 301,
        "white-king": lambda: 302,
        "white-knight": lambda: 303,
        "white-pawn": lambda: 304,
        "white-queen": lambda: 305,
        "white-rook": lambda: 306,
    }
    return cases.get(piece, lambda: "error from function piece2Num -> unknow piece")()
def num2PieceString(piece):
    cases = {
        201: lambda: "black-bishop",
        202: lambda: "black-king",
        203: lambda: "black-knight",
        204: lambda: "black-pawn",
        205: lambda: "black-queen",
        206: lambda: "black-rook",
        301: lambda: "white-bishop",
        302: lambda: "white-king",
        303: lambda: "white-knight",
        304: lambda: "white-pawn",
        305: lambda: "white-queen",
        306: lambda: "white-rook",
    }
    return cases.get(piece, lambda: "error from function piece2Num -> unknow piece")()


In [ ]:
def convert2Xaxis(letter):
    cases = {
        "a": lambda: 7,
        "b": lambda: 6,
        "c": lambda: 5,
        "d": lambda: 4,
        "e": lambda: 3,
        "f": lambda: 2,
        "g": lambda: 1,
        "h": lambda: 0,
    }
    return cases.get(letter, lambda: "error from function convert2Xaxis")()

In [ ]:
def handleSingleInput(data):
    tmp = np.full((8, 8), 7)
    for i in range(len(data)):
        if (data[i][1] == None):
            continue
        else:
            tmp[int(data[i][1][1])-1][convert2Xaxis(data[i][1][0])] = piece2Num(data[i][0])
    return tmp


In [ ]:
def handleMultipleInput(dataArray):
    result = []
    for i in range(len(dataArray)):
        result.append(handleSingleInput(dataArray[i]))
    return result
    

In [ ]:
def handleduplicateLabel(dataArray):
    result = []
    for i in range(len(dataArray)):
        data = dataArray[i]
        tmp = dict()
        detected_chesses = []
        for j in range(len(data)):
            chess_name = data[j][0]
            label = data[j][1]
            confidence_score = data[j][2]
            if (label not in tmp):
                tmp[label] = (confidence_score, chess_name)
                detected_chesses.append((chess_name, label))
            else:
                if (tmp[label][0] < confidence_score):
                    detected_chesses.remove((tmp[label][1], label))
                    detected_chesses.append((chess_name, label))
                    
        result.append(detected_chesses)
    return result

In [ ]:
def getAmountChessPiece(dataArray):
    result = []
    for i in range(len(dataArray)):
        c = 0
        for j in range(8):
            for k in range(8):
                if (dataArray[i][j][k] != 7):
                    c += 1
        result.append(c)
    return result

In [ ]:
def convert2LetterBoard(i):
    cases = {
        7: lambda: "a",
        6: lambda: "b",
        5: lambda: "c",
        4: lambda: "d",
        3: lambda: "e",
        2: lambda: "f",
        1: lambda: "g",
        0: lambda: "h",
    }
    return cases.get(i, lambda: "error from function convert2LetterBoard")()

In [ ]:
def num2piece(num):
    cases = {
        201: lambda: "B",
        202: lambda: "K",
        203: lambda: "N",
        204: lambda: "",
        205: lambda: "Q",
        206: lambda: "R",
        301: lambda: "B",
        302: lambda: "K",
        303: lambda: "N",
        304: lambda: "",
        305: lambda: "Q",
        306: lambda: "R",
        7: lambda: "wtf why empty here!",
    }
    return cases.get(num, lambda: "error from function num2piece -> unknow number")()

In [ ]:
def checkDiffArray(array1, array2):
    out1 = list(set(array1)-set(array2))
    out2 = list(set(array2)-set(array1))
    if (len(out1) == 0):
        return ('-', '-')
    if (len(out2) == 0):
        return ('-', '-')
    return (out1[0], out2[0])

In [ ]:
def findPrevAndPresent(m, n, piece, cleanData):
    prev = []
    next = []
    for i in range(len(cleanData[m])):
        if (cleanData[m][i][0] == num2PieceString(piece)):
            prev.append(cleanData[m][i][1])
    for i in range(len(cleanData[n])):
        if (cleanData[n][i][0] == num2PieceString(piece)):
            next.append(cleanData[n][i][1])
    # print(prev)
    # print(next)
    return checkDiffArray(prev, next)

In [ ]:
def validMove(moved_from, moved_to, piece, move_type):
    if (len(moved_from) < 2 or len(moved_to) < 2): return False
    x1 = convert2Xaxis(moved_from[0])
    y1 = int(moved_from[1])
    x2 = convert2Xaxis(moved_to[0])
    y2 = int(moved_to[1])
    king_move = [(1,0), (-1,0), (0,1), (0,-1), (1,1), (-1,-1), (1,-1), (-1,1)]
    horse_move = [(-2,1), (-2,-1), (2,1), (2,-1), (1,2), (1,-2), (-1, 2), (-1,-2)]

    if (piece % 100 == 4) : # check pawn
        if (piece // 100 == 2) : #black
            if (move_type) : #walk
                if (moved_to == moved_from[0] + str(int (moved_from[1]) - 1) or moved_to == moved_from[0] + str(int (moved_from[1]) - 2)) :
                    return True
                else :  return False
            else :
                return True

        elif (piece // 100 == 3) : #white
            if (move_type) : #walk
                # print(moved_from[1])
                # print(str(int(moved_from[1]) - 1))
                if (moved_to == moved_from[0] + str(int(moved_from[1]) + 1) or moved_to == moved_from[0] + str(int (moved_from[1]) + 2)) :
                    return True
                else : return False
    
    elif (piece % 100 == 1) : # check bishop
        if (abs(x1 - x2) == abs(y1 - y2)) :
            return True
        else : return False
    
    elif (piece % 100 == 6) : # check rook
        if (x1 == x2 or y1 == y2) :
            return True
        else : return False

    elif (piece % 100 == 5) : # check queen
        if (x1 == x2 or y1 == y2 or (abs(x1 - x2) == abs(y1 - y2))) :
            return True
        else : return False
    
    elif (piece % 100 == 2) :#check king
        for pos in king_move :
            if (x1 + pos[0] == x2 and y1 + pos[1] == y2) : return True
        return False
    
    elif (piece % 100 == 3) : # check horse
        for pos in horse_move :
            if (x1 + pos[0] == x2 and y1 + pos[1] == y2) : return True
        return False

    return False

In [ ]:
def lastResult(testSet):
    result = []
    c = 0
    for m in range(len(testSet)-1):
        n = m + 1
        testcaseM = testSet[m]
        testcaseN = testSet[n]
        for i in range(8):
            for j in range(8):
                if (testcaseM[i][j] == testcaseN[i][j]):
                    continue 
                # เดินปกติ
                if (testcaseM[i][j] == 7 and testcaseN[i][j] != 7):
                    # check pawn and Bishop
                    ##################################
                    if (testcaseN[i][j] % 10 == 4 or testcaseN[i][j] % 10 == 1):
                    #####################################
                        move = findPrevAndPresent(m, n, testcaseN[i][j], cleanData)
                        # print(move[0], move[1], testcaseN[i][j], convert2LetterBoard(j) ,str(i+1), testcaseM[i][j], testcaseN[i][j], m ,n)
                        if (validMove(move[0], move[1], testcaseN[i][j], 1)):
                            if (c == 0 and (testcaseN[i][j]//100) % 2 == 0):
                                result.append('..')
                            result.append(num2piece(testcaseN[i][j]) + convert2LetterBoard(j) + str(i+1))
                    #########################################
                    else:
                        if (c == 0 and (testcaseN[i][j]//100) % 2 == 0):
                            result.append('..')
                        result.append(num2piece(testcaseN[i][j]) + convert2LetterBoard(j) + str(i+1))
                    c += 1
                    ########################################################
                    # print("move first condition:", testcaseN[i][j], "at position:", convert2LetterBoard(j), i+1)
                # โดนกิน
                if (lenData[m] == lenData[n]): continue

                if (testcaseM[i][j] != 7 and testcaseN[i][j] != 7 and testcaseM[i][j] != testcaseN[i][j]):
                    # check pawn and Bishop
                    ###############################
                    if (testcaseN[i][j] % 10 == 4 or testcaseN[i][j] % 10 == 1):
                    #################################
                        move = findPrevAndPresent(m, n, testcaseN[i][j], cleanData)
                        # print(move[0], move[1], testcaseN[i][j])
                        if (validMove(move[0], move[1], testcaseN[i][j], 0)):
                            if (c == 0 and (testcaseN[i][j]//100) % 2 == 0):
                                result.append('..')
                            if (testcaseN[i][j] % 10 == 4):
                                if (i-1>=0 and j-1>=0 and testcaseM[i-1][j-1] % 10 == 4 and testcaseN[i-1][j-1] == 7):
                                    result.append(convert2LetterBoard(j-1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                                elif (i-1>=0 and j+1<=7 and testcaseM[i-1][j+1] % 10 == 4 and testcaseN[i-1][j+1] == 7):
                                    result.append(convert2LetterBoard(j+1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                                elif (i+1<=7 and j-1>=0 and testcaseM[i+1][j-1] % 10 == 4 and testcaseN[i+1][j-1] == 7):
                                    result.append(convert2LetterBoard(j-1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                                elif (i+1<=7 and j+1<=7 and testcaseM[i+1][j+1] % 10 == 4 and testcaseN[i+1][j+1] == 7):
                                    result.append(convert2LetterBoard(j+1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                            else :
                                result.append(num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                    #####################
                    else:
                        move = findPrevAndPresent(m, n, testcaseN[i][j], cleanData)
                        # print(move[0], move[1], testcaseN[i][j], convert2LetterBoard(j) ,str(i+1), testcaseM[i][j], testcaseN[i][j], m ,n)
                        if (c == 0 and (testcaseN[i][j]//100) % 2 == 0):
                            result.append('..')
                        if (testcaseN[i][j] % 10 == 4):
                            if (i-1>=0 and j-1>=0 and testcaseM[i-1][j-1] % 10 == 4 and testcaseN[i-1][j-1] == 7):
                                result.append(convert2LetterBoard(j-1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                            elif (i-1>=0 and j+1<=7 and testcaseM[i-1][j+1] % 10 == 4 and testcaseN[i-1][j+1] == 7):
                                result.append(convert2LetterBoard(j+1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                            elif (i+1<=7 and j-1>=0 and testcaseM[i+1][j-1] % 10 == 4 and testcaseN[i+1][j-1] == 7):
                                result.append(convert2LetterBoard(j-1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                            elif (i+1<=7 and j+1<=7 and testcaseM[i+1][j+1] % 10 == 4 and testcaseN[i+1][j+1] == 7):
                                result.append(convert2LetterBoard(j+1) + num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                        else :
                            result.append(num2piece(testcaseN[i][j]) + 'x' + convert2LetterBoard(j) + str(i+1))
                    ########################
                    c += 1
                    # print("move second condition", testcaseN[i][j], "at position:", convert2LetterBoard(j), 'x', i+1)

    return result

In [ ]:
def formatOutput(list) :
  i = 2
  out = '1. '
  for j in range(len(list)):
    out += list[j] + ' '
    if (j % 2 == 1 and j != len(list) - 1):
      out += str(i) + '. '
      i += 1
  return out

In [ ]:
def unsharp_mask(image, kernel_size=(5, 5), alpha=1.5, beta=-0.5, gamma=0):
    # Blur the image
    blurred = cv2.GaussianBlur(image, kernel_size, 0)
    
    # Combine original and blurred images
    sharpened = cv2.addWeighted(image, alpha, blurred, beta, gamma)
    return sharpened

def get_largest_connected_component(edges): #mai dai chai
    # Find contours from the edges
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No contours found.")
        return None, np.zeros_like(edges)

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Compute the convex hull of the largest contour
    convex_hull = cv2.convexHull(largest_contour)

    # Create a blank mask
    mask = np.zeros_like(edges)

    # Draw the largest contour on the mask
    cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
    # cv2.drawContours(mask, [convex_hull], -1, 255, thickness=1)

    return largest_contour, mask

def canny_edge(image, lower_th, upper_th):
    median = np.median(image)  # Calculate median pixel value
    # print("median", median)
    adjust = 0.5
    lower_threshold = max(0, (1.0 + lower_th ) * median)  # Lower threshold
    upper_threshold = min(255, (1.0 + upper_th) * median)  # Upper threshold

    # print("lower_th", lower_threshold)
    # print("upper_th", upper_threshold)

    edges = cv2.Canny(image, lower_threshold, upper_threshold)

    return edges

def select_connected_edges(edges):
    # Apply morphological closing to close small gaps between edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # Adjust kernel size as needed
    closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Find contours of the connected components
    contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank mask for the selected edges
    selected_edges = np.zeros_like(edges)

    # Iterate over the contours and select large connected components
    big_contours = []

    for contour in contours:
        if cv2.contourArea(contour) > 10000:  # Adjust area threshold as needed
            # Draw the contour on the selected edges mask
            # print("area", cv2.contourArea(contour))
            big_contours.append(contour)

    # print("big contour", len(big_contours))
    largest_contour = max(big_contours, key=cv2.contourArea)
    cv2.drawContours(selected_edges, [largest_contour], -1, 255, thickness=1)

    return selected_edges

def detect_chessboard_with_hough(image):
    """
    Detects the edges of a chessboard using the Hough Line Transform.
    """
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lower_bound = 0
    upper_bound = 20
    new_value = 255

    mask = cv2.inRange(image, lower_bound, upper_bound)

    image[mask > 0] = new_value
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    dummy = image.copy()

    # Apply sharpening
    sharpened_image = unsharp_mask(blurred)

    #-----------------------------------------------------------------

    # cv2.imshow("sharpened", sharpened_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    #-----------------------------------------------------------------

    # Use Canny edge detection
    edges = canny_edge(sharpened_image, -0.4, -0.2)

    #-----------------------------------------------------------------

    # cv2.imshow("canny edge", edges)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    #-----------------------------------------------------------------

    # largest_contour, largest_component_mask = get_largest_connected_component(edges)

    selected_closed_edges = select_connected_edges(edges)

    #-----------------------------------------------------------------

    # cv2.imshow("selected closed edge", selected_closed_edges)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    #-----------------------------------------------------------------

    # Use Hough Line Transform to detect lines
    lines = cv2.HoughLinesP(selected_closed_edges, rho=1, theta=np.pi / 180, threshold=100, minLineLength=30, maxLineGap=120)

    # print("len(lines)", len(lines))

    if lines is not None:
        # Convert lines to a set of endpoints
        lines = lines[:, 0]  # Remove the unnecessary dimension
        image_with_lines = image.copy()
        
        # Draw detected lines
        for x1, y1, x2, y2 in lines:
            cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Find quadrilateral (chessboard edges) from lines
        chessboard_edges = find_chessboard_edges(lines, image.shape)

        if chessboard_edges is not None:
            # Draw the detected chessboard edges
            for (x1, y1), (x2, y2) in zip(chessboard_edges, chessboard_edges[1:] + [chessboard_edges[0]]):
                cv2.line(image_with_lines, (x1, y1), (x2, y2), (255, 0, 0), 3)

        return image_with_lines, chessboard_edges

    return image, []

def are_sides_parallel(points, tolerance=0.5):
    # Calculate slopes of sides
    slopes = []
    for i in range(4):
        x1, y1 = points[i]
        x2, y2 = points[(i + 1) % 4]  # Next point
        if x2 - x1 != 0:  # Avoid division by zero
            slopes.append((y2 - y1) / (x2 - x1))
        else:
            slopes.append(float('inf'))  # Vertical line

    # Check if opposite sides are parallel (slopes are approximately equal)
    # print("slope 1", abs(slopes[0] - slopes[2]))
    # print("slope 2", abs(slopes[1] - slopes[3]))
    return abs(slopes[0] - slopes[2]) < tolerance and abs(slopes[1] - slopes[3]) < tolerance

def find_chessboard_edges(lines, image_shape): # have problem with convex hull
    """
    Filters lines to find the chessboard edges by looking for four lines
    that form a quadrilateral around the chessboard.
    """
    # Convert lines into a set of points
    points = []
    for x1, y1, x2, y2 in lines:
        points.append((x1, y1))
        points.append((x2, y2))

    points = np.array(points)

    # Find a convex hull of the points
    hull = cv2.convexHull(points)
    if len(hull) < 4:
        return None  # Not enough points to form a quadrilateral

    # Approximate the convex hull to a quadrilateral
    epsilon = 0.01 * cv2.arcLength(hull, True)
    approx = cv2.approxPolyDP(hull, epsilon, True)

    # if len(approx) == 4 :
    if len(approx) == 4 and are_sides_parallel([(pt[0][0], pt[0][1]) for pt in approx]):
        return [(pt[0][0], pt[0][1]) for pt in approx]  # Return the points of the quadrilateral

    return None

In [ ]:
import cv2
import numpy as np
def detect_chessboard_edges(frame):
    """
    Detect chessboard edges in the frame using Hough Lines and return the edges and mask.
    """
    height, width = frame.shape[:2]

    # Define ROI (optional: adjust margins based on expected chessboard location)
    margin = 50
    roi = frame[margin:height - margin, margin:width - margin]

    # Convert to grayscale and apply edge detection
    lower_bound = 0
    upper_bound = 10
    new_value = 255
    mask = cv2.inRange(frame, lower_bound, upper_bound)
    frame[mask > 0] = new_value
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    sharpened_image = unsharp_mask(blurred)
    edges = canny_edge(sharpened_image, -0.4, -0.2)
    edges = select_connected_edges(edges)  # Custom function to select the largest connected edges

    # Morphological operations to close gaps in edges
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)


    scale_width = 800 / closed_edges.shape[1]
    scale_height = 600 / closed_edges.shape[0]
    scale = min(scale_width, scale_height)
    resized_frame = cv2.resize(closed_edges, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    # cv2.imshow("closed_edges", resized_frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows

    # Find contours
    contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours by area (remove small or noisy contours)
    filtered_contours = [c for c in contours if cv2.contourArea(c) > 10000]

    # Create a blank mask and draw the largest contour (assume it's the chessboard)
    mask = np.zeros_like(edges)
    largest_contour = None
    if filtered_contours:
        largest_contour = max(filtered_contours, key=cv2.contourArea)
        cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

    # Adjust the detected contour to match the full frame (consider the ROI margins)
    board_edges = np.zeros_like(frame)
    if filtered_contours:
        largest_contour = np.array(largest_contour)
        # largest_contour = np.array(largest_contour) + [margin, margin] # use roi
        cv2.drawContours(board_edges, [largest_contour], -1, (0, 255, 0), 2)

    # Apply Hough Line Detection to the closed edges
    lines = cv2.HoughLinesP(closed_edges, rho=1, theta=np.pi / 180, threshold=100, minLineLength=150, maxLineGap=120) # best (1, 180, 100, 150, 120) ,(100, 500, 500)

    chessboard_edges = find_chessboard_edges(lines[0], frame.shape)

    #-------------------------------------------v

    # image_with_lines = frame.copy()
    # if chessboard_edges is not None:
    # # Draw the detected chessboard edges
    #     for (x1, y1), (x2, y2) in zip(chessboard_edges, chessboard_edges[1:] + [chessboard_edges[0]]):
    #         cv2.line(image_with_lines, (x1, y1), (x2, y2), (255, 0, 0), 3)

    # return image_with_lines, chessboard_edges

    #-------------------------------------------^

    if lines is not None:
        points = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        points.append((x1, y1))
        points.append((x2, y2))
        # Draw the lines on the board edges (optional visualization)
        cv2.line(board_edges, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # Convert points to a NumPy array
    points = np.array(points)

    # Find the convex hull around the points
    hull = cv2.convexHull(points)

    # Approximate the convex hull to a quadrilateral
    epsilon = 0.02 * cv2.arcLength(hull, True)  # Adjust epsilon as needed
    approx = cv2.approxPolyDP(hull, epsilon, True)

    # Ensure the approximation is a rectangle or square (4 points)
    if len(approx) == 4:
        # Draw the rectangle or square on the board edges for visualization
        cv2.polylines(board_edges, [approx], isClosed=True, color=(0, 255, 255), thickness=2)

        # Optionally, you can extract the points as the final rectangle
        rect_points = [tuple(pt[0]) for pt in approx]

    return rect_points


In [ ]:
def distance_squared(point):
    x, y = point
    return x**2 + y**2

In [ ]:
import cv2
import os
import numpy as np

def rotate_video_swap_dimensions(video_path, rotate_keyword='rotate', output_suffix='_rotated'):
    """
    Rotates the video 90 degrees clockwise and swaps its width and height if the file path contains the specified keyword.
    
    Parameters:
        video_path (str): Path to the original video file.
        rotate_keyword (str): Keyword to check in the file path to decide rotation.
        output_suffix (str): Suffix to append to the original filename for the rotated video.
        
    Returns:
        str: Path to the rotated video if rotation was performed, else the original video path.
    """
    
    # Check if the specified keyword is in the file path (case-insensitive)
    if rotate_keyword in video_path:
        # Generate the rotated video path by appending the suffix before the file extension
        base, ext = os.path.splitext(video_path)
        rotated_video_path = "/kaggle/working/2_Move_rotate_student_rotated.mp4"
        
        # Open the original video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise IOError(f"Cannot open video file: {video_path}")
        
        # Retrieve original video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Define the codec and create VideoWriter object with swapped dimensions
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec if needed
        out = cv2.VideoWriter(rotated_video_path, fourcc, fps, (original_height, original_width))
        
        if not out.isOpened():
            cap.release()
            raise IOError(f"Cannot write to video file: {rotated_video_path}")
        
        print(f"Starting rotation of video: {video_path}")
        print(f"Original resolution: {original_width}x{original_height}")
        print(f"Rotated resolution: {original_height}x{original_width}")
        print(f"Total frames to process: {frame_count}")
        
        frame_idx = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break  # End of video
            
            # Rotate the frame 90 degrees clockwise
            # OpenCV's rotate function changes the resolution accordingly
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
            
            # Write the rotated frame to the output video
            out.write(rotated_frame)
            
            frame_idx += 1
            if frame_idx % 100 == 0 or frame_idx == frame_count:
                print(f"Processed {frame_idx}/{frame_count} frames...")
        
        # Release resources
        cap.release()
        out.release()
        
        print(f"Rotation complete. Rotated video saved at: {rotated_video_path}")
        
        return rotated_video_path
    else:
        print(f"No rotation needed for video: {video_path}")
        return video_path




In [ ]:
import cv2
import mediapipe as mp

def detectHand(frame, hand_detector):
    # Convert the BGR image to RGB.
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame and detect hands.
    results = hand_detector.process(rgb_frame)
    
    # Check if any hands are detected.
    if results.multi_hand_landmarks:
        return True
    return False

def locateChess(vids, chessboardLabel):
    """
    Processes a video to locate chessboards, optionally rotating frames if 'rotate' is in the filename.

    Parameters:
    - vids (str): Path to the video file.
    - chessboardLabel (str): Label for the chessboard (used in YOLO processing).
    - rotation_angle (float, optional): Angle in degrees to rotate frames. If None, uses 90 degrees clockwise.

    Returns:
    - list: Results from processing frames.
    """
    # Initialize MediaPipe Hands.
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,       # Video stream, so set to False.
        max_num_hands=2,               # Maximum number of hands to detect.
        min_detection_confidence=0.7,  # Minimum confidence for detection.
        min_tracking_confidence=0.7    # Minimum confidence for tracking.
    )
    
    cap = cv2.VideoCapture(vids)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video file {vids}")
        return []
    
    # Determine if rotation is needed based on the video file name
    filename = os.path.basename(vids)
    should_rotate = "rotate" in filename
    
    # Retrieve frames per second (FPS) of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        print("Error: Cannot retrieve FPS from the video.")
        cap.release()
        return []
    
    # Calculate the frame interval for approximately every 3 seconds
    interval = int(fps * 9)  # For every ~3 seconds
    if interval <= 0:
        print("Error: Invalid frame interval calculated.")
        cap.release()
        return []
    
    # Calculate the starting frame at 1 second
    start_frame = int(fps * 1)  # 1 second into the video
    if start_frame < 0:
        print("Error: Invalid start frame calculated.")
        cap.release()
        return []
    
    print(f"Video FPS: {fps}")
    print(f"Starting processing at frame {start_frame} (1 second).")
    print(f"Processing every {interval} frames (approximately every 3 seconds).")
    
    results = []
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            # End of video
            break

        if frame_count >= start_frame and (frame_count - start_frame) % interval == 0:
            # Check for hand presence
            hands_detected = detectHand(frame, hands)
            if hands_detected:
                print(f"Skipped frame {frame_count} (Hands detected).")
            else:
                # Process the captured frame
                frame_result = testYOLO(frame, chessboardLabel,should_rotate)
                results.append(frame_result)
                current_time = frame_count / fps
                print(f"Processed frame {frame_count} (Time: {current_time:.2f}s): {frame_result}")
        
        frame_count += 1
    
    # Release the video capture and MediaPipe resources
    cap.release()
    hands.close()
    print("All results:", results)
    
    return results

# Example usage:
# results = locateChess('path_to_video.mp4', 'chessboard')


In [ ]:
import numpy as np

def select_points_swapped(
    merged_points,
    starting_coord,
    total_points=81,
    points_per_column=9,
    x_threshold=40,
    y_spacing=10,
    delta_y=50
):
    """
    Selects points grouped into columns based on x-coordinates.

    Parameters:
    - merged_points: List of tuples or arrays representing points (x, y).
    - starting_coord: Tuple or array representing the starting coordinate (x, y).
    - total_points: Total number of points to select.
    - points_per_column: Maximum number of points to select per column.
    - x_threshold: Threshold to define the x range for a column.
    - y_spacing: Minimum spacing between selected points in the y-direction.
    - delta_y: (Unused in original function; included for consistency)

    Returns:
    - selected_points: List of selected points as tuples.
    """
    selected_points = []

    # Convert list to NumPy array for easier processing
    points_array = np.array(merged_points)

    # Sort points by x-coordinate (ascending)
    sorted_indices = np.argsort(points_array[:, 1])
    sorted_points = points_array[sorted_indices]

    # Initialize variables
    current_start = np.array(starting_coord)
    points_remaining = total_points

    while points_remaining > 0 and len(sorted_points) > 0:
        # Compute distances to the current_start point
        distances = np.linalg.norm(sorted_points - current_start, axis=1)
        nearest_idx = np.argmin(distances)
        nearest_point = sorted_points[nearest_idx]
        print("near", nearest_point)

        # If the nearest point is too far left, remove points to the left
        if nearest_point[0] < starting_coord[0] - x_threshold:
            # Remove points with x < starting_coord[0] - x_threshold
            sorted_points = sorted_points[sorted_points[:, 0] >= starting_coord[0] - x_threshold]
            if len(sorted_points) == 0:
                break
            continue        

        # Define the x range for the current column
        x_min = nearest_point[0] - x_threshold
        x_max = nearest_point[0] + x_threshold

        # Select points within this column
        column_mask = (sorted_points[:, 0] >= x_min) & (sorted_points[:, 0] <= x_max)
        column_points = sorted_points[column_mask]

        if len(column_points) == 0:
            break  # No more points to select

        # Extract the y-coordinate of the nearest point
        y_nearest = nearest_point[1]

        # **New Step:** Filter points to have y >= y_nearest
        column_points = column_points[column_points[:, 1] >= y_nearest]

        if len(column_points) == 0:
            # If no points satisfy y >= y_nearest, skip to next column
            sorted_points = sorted_points[~column_mask]
            current_start = np.array([x_max + 1, current_start[1]])  # Move to the right of the current column
            continue

        # Sort column points by y-coordinate (ascending)
        column_sorted_indices = np.argsort(column_points[:, 1])
        column_sorted_points = column_points[column_sorted_indices]

        last_selected_y = None

        # Select the first 'points_per_column' points, ensuring y difference > y_spacing
        selected_column_points = []
        for pt in column_sorted_points:
            if last_selected_y is None or abs(pt[1] - last_selected_y) > y_spacing:
                selected_column_points.append(pt)
                last_selected_y = pt[1]
                points_remaining -= 1
                if points_remaining == 0 or len(selected_column_points) == points_per_column:
                    break

        # Add the selected column points to the overall selected points
        for pt in selected_column_points:
            selected_points.append(tuple(pt))

        # Remove selected points from sorted_points
        sorted_points = sorted_points[~column_mask]

        # Update current_start for the next column (move to the right of the current column)
        current_start = np.array([x_max + 1, current_start[1]])

    return selected_points


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import string

def create_chessboard_labels(grid_points,rotate=False):
        # Sort grid points into rows based on y-coordinates
        # grid_points = sorted(grid_points, key=lambda p: (-p[1], p[0]))  # Sort by y descending, then x ascending
        if(rotate):
            grid_mod_9 = [grid_points[i::9] for i in range(9)]
            grid_points = [item for sublist in grid_mod_9 for item in sublist]
            print("Grid",grid_points)

        rows = [grid_points[i:i + 9] for i in range(0, len(grid_points), 9)]
        
        labels = {}
        total_rows = len(rows) - 1  # Number of squares vertically

        for row_index in range(total_rows):
            for col_index in range(8):  # 8 squares horizontally
                # Columns labeled from 'h' to 'a' (left to right)
                column_label = chr(104 - col_index)  # 'h' (104) to 'a' (97)

                # Rows labeled from '1' to '8' (top to bottom)
                row_label = row_index + 1  # '1' is top row, '8' is bottom row
                label = f"{column_label}{row_label}"

                # Define square boundaries using four points
                square = {
                    "top_left": rows[row_index][col_index],
                    "top_right": rows[row_index][col_index + 1],
                    "bottom_left": rows[row_index + 1][col_index],
                    "bottom_right": rows[row_index + 1][col_index + 1],
                }
                labels[label] = square

        # Debug: Print all labels with their boundaries
        # for label, square in labels.items():
        #     print(f"{label}: {square}")
        # print(labels)
        return labels

def get_chessboard_label(x, y, chessboard_labels):
        for label, square in chessboard_labels.items():
        # Extract all x and y coordinates to determine boundaries
            x_coords = [square["top_left"][0], square["top_right"][0],
                        square["bottom_left"][0], square["bottom_right"][0]]
            y_coords = [square["top_left"][1], square["top_right"][1],
                        square["bottom_left"][1], square["bottom_right"][1]]
            
            x_min = min(x_coords)
            x_max = max(x_coords)
            y_min = min(y_coords)
            y_max = max(y_coords)
            
            # Debug: Print boundaries and point
            # print(f"Checking {label}: x[{x_min}, {x_max}], y[{y_min}, {y_max}] with point ({x}, {y})")
            
            if (x_min <= x <= x_max) and (y_min <= y <= y_max):
                return label
        return None



def select_points(merged_points, starting_coord, total_points=81, points_per_row=9, y_threshold=10,x_spacing=10, delta_x=50):
    selected_points = []

    # Convert list to NumPy array for easier processing
    points_array = np.array(merged_points)

    # Sort points by y-coordinate (ascending)
    sorted_indices = np.argsort(points_array[:, 1])
    sorted_points = points_array[sorted_indices]

    # Initialize variables
    current_start = np.array(starting_coord)
    points_remaining = total_points

    while points_remaining > 0 and len(sorted_points) > 0:
        # Compute distances to the current_start point
        distances = np.linalg.norm(sorted_points - current_start, axis=1)
        nearest_idx = np.argmin(distances)
        nearest_point = sorted_points[nearest_idx]
        print("near",nearest_point)

        if nearest_point[1] < starting_coord[1]-y_threshold:
            # Remove points with y < starting_coord[1]
            sorted_points = sorted_points[sorted_points[:, 1] >= starting_coord[1]-y_threshold]
            if len(sorted_points) == 0:
                break
            continue        


        # Define the y range for the current row
        y_min = nearest_point[1] -  y_threshold
        y_max = nearest_point[1] +  y_threshold

        # Select points within this row
        row_mask = (sorted_points[:, 1] >= y_min) & (sorted_points[:, 1] <= y_max)
        row_points = sorted_points[row_mask]

        if len(row_points) == 0:
            break  # No more points to select

        # Extract the x-coordinate of the nearest point
        x_nearest = nearest_point[0]

        # **New Step:** Filter points to have x >= x_nearest
        row_points = row_points[row_points[:, 0] >= x_nearest]

        if len(row_points) == 0:
            # If no points satisfy x >= x_nearest, skip to next row
            sorted_points = sorted_points[~row_mask]
            current_start = np.array([current_start[0], y_max + 1])  # Move below the current row
            continue

        # Sort row points by x-coordinate (ascending)
        row_sorted_indices = np.argsort(row_points[:, 0])
        row_sorted_points = row_points[row_sorted_indices]

        last_selected_x = None

        # Select the first 'points_per_row' points, ensuring x difference > 20
        selected_row_points = []
        for pt in row_sorted_points:
            if last_selected_x is None or abs(pt[0] - last_selected_x) > 10:
                selected_row_points.append(pt)
                last_selected_x = pt[0]
                points_remaining -= 1
                if points_remaining == 0 or len(selected_row_points) == points_per_row:
                    break

        # Add the selected row points to the overall selected points
        for pt in selected_row_points:
            selected_points.append(tuple(pt))

        # Remove selected points from sorted_points
        sorted_points = sorted_points[~row_mask]

        # Update current_start for the next row (move below the current row)
        current_start = np.array([current_start[0], y_max + 1])

    return selected_points


# Line detection function
def detect_lines(image, starting_coord=(147, 538), border=[],rotated=False):
    # Convert to grayscale and apply Gaussian blur
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Adjusted Hough Line Transform parameters
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi / 180, threshold=100, minLineLength=30, maxLineGap=120)

    def extend_line(x1, y1, x2, y2, img_width, img_height):
        if x1 == x2:  # Vertical line
            return x1, 0, x2, img_height
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1

        # Calculate intersection points with image borders
        y_at_x0 = intercept
        y_at_xmax = slope * img_width + intercept
        x_at_y0 = -intercept / slope
        x_at_ymax = (img_height - intercept) / slope

        # Determine the two points where the line intersects the image edges
        points = []
        if 0 <= y_at_x0 <= img_height:
            points.append((0, int(y_at_x0)))
        if 0 <= y_at_xmax <= img_height:
            points.append((img_width, int(y_at_xmax)))
        if 0 <= x_at_y0 <= img_width:
            points.append((int(x_at_y0), 0))
        if 0 <= x_at_ymax <= img_width:
            points.append((int(x_at_ymax), img_height))

        # Return the first two points as the extended line
        return points[0][0], points[0][1], points[1][0], points[1][1]

    img_height, img_width = image.shape[:2]
    extended_lines = []
    vertical_lines = []
    horizontal_lines = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1_ext, y1_ext, x2_ext, y2_ext = extend_line(x1, y1, x2, y2, img_width, img_height)
            extended_lines.append((x1_ext, y1_ext, x2_ext, y2_ext))

            # Check for vertical or horizontal lines
            slope = (y2 - y1) / (x2 - x1) if x2 != x1 else float('inf')  # Handle vertical lines
            if abs(slope) > 1:  # Horizontal lines (near zero slope)
                horizontal_lines.append((x1_ext, y1_ext, x2_ext, y2_ext))
            else:
                vertical_lines.append((x1_ext, y1_ext, x2_ext, y2_ext))

            # Draw each extended line on the image
            cv2.line(image, (x1_ext, y1_ext), (x2_ext, y2_ext), (0, 255, 0), 2)  # Green line for detection

    # Function to find the intersection of two lines
    def intersection(line1, line2):
        x1, y1, x2, y2 = map(float, line1)
        x3, y3, x4, y4 = map(float, line2)

        # Calculate determinant
        det = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
        if abs(det) == 0:  # Parallel lines
            return None

        # Calculate intersection point
        px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / det
        py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / det
        return (int(px), int(py))

    # Function to merge close points
    def merge_close_points(points, threshold=15):
        merged_points = []
        for point in points:
            if not any(np.linalg.norm(np.array(point) - np.array(existing_point)) < threshold for existing_point in merged_points):
                merged_points.append(point)
        return merged_points

    # Find intersections only between horizontal and vertical lines
    intersection_points = []
    for v_line in vertical_lines:
        for h_line in horizontal_lines:
            point = intersection(v_line, h_line)
            if point:
                intersection_points.append(point)

    # Merge close points and filter valid points
    merged_points = merge_close_points(intersection_points, threshold=25)
    print(merged_points)

    # Draw filtered points on the image
    # for point in merged_points:
    #     cv2.circle(image, point, 6, (0, 0, 255), -1)  # Red circle for intersections
    y_coords = [point[1] for point in border]
    # Calculate the absolute differences
    print("Y",y_coords)
    diff_1 = abs(y_coords[0] - y_coords[1])
    diff_2 = abs(y_coords[2] - y_coords[3])
    # Find the maximum difference
    max_diff = max(diff_1, diff_2)
    print(max_diff)

    x_coords = [point[0] for point in border]
    print("X Coordinates:", x_coords)

    # Calculate the absolute differences for x-coordinates
    diff_x1 = abs(x_coords[0] - x_coords[1])
    diff_x2 = abs(x_coords[2] - x_coords[3])

    # Find the maximum difference among x-coordinates
    max_diff_x = max(diff_x1, diff_x2)
    print("Maximum X Difference:", max_diff_x)

    if(rotated):
        # merged_points = [(y, x) for (x, y) in merged_points]
        selected_points = select_points_swapped(merged_points, starting_coord,x_threshold=max_diff_x)
    else:
        selected_points = select_points(merged_points, starting_coord, total_points=81, points_per_row=9, y_threshold=max_diff)
    print("checkMergePoint",merged_points)
    print('start',starting_coord)
    print(selected_points)
    chessboard_labels = create_chessboard_labels(selected_points,rotated)

    # Test the function with an example coordinate

    print(selected_points)
    for point in selected_points:
        cv2.circle(image, point, 6, (0, 0, 255), -1)  # Red circle for intersections
        # Prepare the coordinate text
        coord_text = f'({point[0]}, {point[1]})'
        # Choose a font scale and thickness
        font_scale = 0.5
        thickness = 1
        # Get the size of the text to create a background rectangle
        (text_width, text_height), baseline = cv2.getTextSize(coord_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
        # Position the text with an offset
        text_x = point[0] + 10
        text_y = point[1] - 10
        # Ensure the text doesn't go outside the image boundaries
        if text_x + text_width > img_width:
            text_x = point[0] - text_width - 10
        if text_y - text_height < 0:
            text_y = point[1] + text_height + 10
        # Draw a filled rectangle as background for better readability
        cv2.rectangle(image, (text_x, text_y - text_height - baseline), 
                             (text_x + text_width, text_y + baseline), 
                             (255, 255, 255), cv2.FILLED)
        # Put the text on top of the rectangle
        cv2.putText(image, coord_text, (text_x, text_y), 
                    cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), thickness, cv2.LINE_AA)

    return image,merged_points,chessboard_labels


# Video processing function
def process_video_with_fixed_board(video_path):
    cap = cv2.VideoCapture(video_path)
    filename = os.path.basename(video_path)
    should_rotate = "rotate" in filename

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    first_frame = True
    processed_first_frame = None
    img = None

    # Get video properties to determine the display size
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    aspect_ratio = width / height
    window_width = 480  # Set a fixed window size width (can be adjusted)
    window_height = int(window_width / aspect_ratio)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or unable to read frame.")
            break
        if first_frame:
            print("Processing the first frame...")
            img = frame.copy()
            borderPoints = detect_chessboard_edges(img)
            sorted_borderPoints = sorted(borderPoints, key=distance_squared)
            # print(sorted_borderPoints)
            processed_first_frame,points,chessboardLabel = detect_lines(frame,sorted_borderPoints[0],sorted_borderPoints,rotated=should_rotate)
            first_frame = False
        # Resize frame while maintaining aspect ratio
        resized_frame = cv2.resize(frame, (window_width, window_height))
        # Resize processed_first_frame to match the resized frame size
        processed_first_frame_resized = cv2.resize(processed_first_frame, (window_width, window_height))
        origin = cv2.resize(frame, (window_width, window_height))
        # Blend the resized frames
        output_frame = cv2.addWeighted(resized_frame, 0.7, processed_first_frame_resized, 0.3, 0)
        # cv2.imshow("Chessboard Tracking", frame)
        # cv2.waitKey(0)
        captured_frame = output_frame.copy()
        break
        # Quit on 'q' key
        if cv2.waitKey(30) & 0xFF == ord("q"):
            break     
    cap.release()
    # cv2.destroyAllWindows()
    return captured_frame,img,points,chessboardLabel

def testYOLO(testImg,chessboard_labels,rotate=False):
    detected_chesses = []  # Initialize the list to store (chessname, label) tuples
    if testImg is not None:
        # Save the processed frame to a temporary file
        temp_image_path = "temp_test_image.jpg"
        cv2.imwrite(temp_image_path, testImg)

        # Load the YOLO model
        model = YOLO("/kaggle/input/chess_detection_x75/pytorch/default/1/chess_model_x_75.pt")  # Use forward slashes for cross-platform compatibility

        # Run prediction on the saved image
        results = model.predict(source=temp_image_path, imgsz=640)

        # Define the class mapping based on your provided class list
        class_to_chess = {
            0: "bishop",
            1: "black-bishop",
            2: "black-king",
            3: "black-knight",
            4: "black-pawn",
            5: "black-queen",
            6: "black-rook",
            7: "white-bishop",
            8: "white-king",
            9: "white-knight",
            10: "white-pawn",
            11: "white-queen",
            12: "white-rook",
        }

        # Iterate through the detection results
        for result in results:
            for box in result.boxes.data.tolist():  # Each box is [x1, y1, x2, y2, confidence, class]
                x1, y1, x2, y2, confidence, cls = box
                x_center = int((x1 + x2) / 2)
                y_center = int(abs(y2-y1)*3/4+y1)

                # Map the class index to a chess piece name
                chess_class = int(cls)
                chess_name = class_to_chess.get(chess_class, "Unknown")
                confidence_score = confidence

                # Map the (x, y) coordinates to a chessboard label
                print(chessboard_labels)
                label = get_chessboard_label(x_center, y_center, chessboard_labels)


                # Append the tuple to the list
                detected_chesses.append((chess_name, label, confidence_score))

                # Optional: Print debug information
                print(f"Detected {chess_name} at {label} with confidence {confidence_score:.2f}")

        # Display the prediction result with bounding boxes
        for result in results:
            annotated_frame = result.plot()  # Annotated image with bounding boxes
            scale_width = 1400 / annotated_frame.shape[1]
            scale_height = 1200 / annotated_frame.shape[0]
            scale = min(scale_width, scale_height)
            resized_frame = cv2.resize(annotated_frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
            # cv2.imshow("<picture_name", resized_frame)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # cv2.imshow("YOLO Detection Result", annotated_frame)

        # Wait for user to close the window
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

    else:
        print("No image was processed.")
    
    print(detected_chesses)

    return detected_chesses

# Main Script
# video_path = r"D:\CP31\DigImProj\chess_dataset\vids\2_Move_rotate_student.mp4"# Replace with your video path
# video_path = r"chess_dataset/vids/2_Move_rotate_student_rotated.mp4"# Replace with your video path
# video_path = r"D:\CP31\DigImProj\chess_dataset\vids\2_Move_rotate_student_rotated.mp4"  # Replace with your video path
# video_path = r"D:\CP31\DigImProj\chess_dataset\vids\6_Move_student.mp4"  # Replace with your video path
# video_path = r"D:\CP31\DigImProj\chess_dataset\vids\8_Move_student.mp4"  # Replace with your video path
# new_video_path=rotate_video_swap_dimensions(video_path)

# Proceed with processing the video
# testImg, origins, points, chessboardLabel = process_video_with_fixed_board(video_path)
# print(points)
# cv2.imshow("Captured First Frame", testImg)
# cv2.waitKey(0)  
# testYOLO(origins,chessboardLabel)
# locateChess(video_path,chessboardLabel)
# data = locateChess(video_path,chessboardLabel)
# tryData = handleMultipleInput(handleduplicateLabel(data))
# output = formatOutput(lastResult(tryData))
# print(output)
     


In [ ]:
import os
import pandas as pd

# Define the directory containing the videos
video_dir = '/kaggle/input/cu-chess-detection/Chess Detection Competition/test_videos'

# Initialize a list to store the results
results = []

# Iterate over each file in the video directory
for filename in os.listdir(video_dir):
    # Check if the file is a video (e.g., ends with .mp4)
    if filename.endswith('.mp4'):
        video_path = os.path.join(video_dir, filename)
        filename = os.path.basename(video_path)
        rotate = "rotate" in filename
        if(rotate):
            video_path=rotate_video_swap_dimensions(video_path)
        # else:
        #     print('skip')
        #     continue
    #-------------------------------------------
        try:
            # Process the video
            result = process_video_with_fixed_board(video_path)
            
            # Check if the function returned valid output
            if result is None:
                raise ValueError(f"process_video_with_fixed_board returned None for video {video_path}")
            
            rigtestImg, oins, points, chessboardLabel = result
            
            data = locateChess(video_path, chessboardLabel)
        
            cleanData = handleduplicateLabel(data)
            tryData = handleMultipleInput(cleanData)
            lenData = getAmountChessPiece(tryData)
            output = formatOutput(lastResult(tryData))
            
            # Append the result to the list
            results.append({
                'row_id': filename,      # Using the video filename as the ID
                'output': output         # The result from lastResult
            })
        
        except Exception as e:
            # Handle any exceptions and optionally log them
            print(f"Error processing {filename}: {e}")
            results.append({
                'row_id': filename,
                'output': f"Error: {e}"
            })

        # print("len: ", len(results))

##BONUS
#===============================================================================================================
bonus_dir="/kaggle/input/cu-chess-detection/Chess Detection Competition/bonus_video/Bonus Long Video Label.mp4"
if result is None:
    raise ValueError(f"process_video_with_fixed_board returned None for video {bonus_dir}")
rigtestImg, oins, points, chessboardLabel = result
data = locateChess(bonus_dir, chessboardLabel)
cleanData = handleduplicateLabel(data)
tryData = handleMultipleInput(cleanData)
lenData = getAmountChessPiece(tryData)
output = formatOutput(lastResult(tryData))
results.append({
    'row_id': '(Bonus)Long_video_student.mp4',
    'output': output
})
#===============================================================================================================

# Create a DataFrame from the results
df = pd.DataFrame(results)
df = df.dropna(how='all')
# Add the custom row
print(df)

# Specify the path where you want to save the CSV
csv_output_path = '/kaggle/working/submission.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_output_path, index=False, mode='w')

# Remove trailing newline by reading the file and rewriting without the final newline
with open(csv_output_path, 'r') as file:
    lines = file.readlines()

# Rewrite the file, excluding the trailing newline in the last line
with open(csv_output_path, 'w') as file:
    file.writelines(lines[:-1] + [lines[-1].rstrip('\n')])

print(f"Results have been saved to {csv_output_path}")